In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from scipy import stats

load_dotenv()  # This loads the variables from .env

db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

In [58]:
query = """
WITH miner_info AS (
    SELECT *
    FROM miners_table
    WHERE uid != '[null]' AND is_current_uid = 'true'
    ORDER BY rank DESC
),
prediction_data AS (
    SELECT 
        pvt.predicted_value_timestamp,
        pvt.minute_offset,
        pt.timestamp,
        pvt.prediction,
        pt.miner_id,
        m.rank AS miner_rank,
        m.uid AS miner_uid,
        m.hot_key,
        m.cold_key
    FROM predictions_values_table pvt
    JOIN predictions_table pt ON pvt.predictions_id = pt.id
    JOIN miner_info m ON pt.miner_id = m.id
    WHERE pt.timestamp >= '2024-09-19 00:00:00'::timestamp
      AND pt.timestamp < CURRENT_DATE + INTERVAL '1 day'
    ORDER BY pt.timestamp DESC
)
SELECT 
    pd.*,
    y.price AS actual_price,
    y.timestamp AS yfinance_timestamp
FROM prediction_data pd
LEFT JOIN LATERAL (
    SELECT price, timestamp
    FROM yfinance_sp500_table
    WHERE timestamp <= pd.predicted_value_timestamp
    ORDER BY timestamp DESC
    LIMIT 1
) y ON true
ORDER BY pd.timestamp DESC;
"""

df = pd.read_sql(query, engine)

# Basic data overview
print(df.info())
print("\nSample data:")
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410346 entries, 0 to 410345
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   predicted_value_timestamp  410346 non-null  datetime64[ns]
 1   minute_offset              410346 non-null  int64         
 2   timestamp                  410346 non-null  datetime64[ns]
 3   prediction                 410346 non-null  float64       
 4   miner_id                   410346 non-null  object        
 5   miner_rank                 410346 non-null  float64       
 6   miner_uid                  410346 non-null  object        
 7   hot_key                    410346 non-null  object        
 8   cold_key                   410346 non-null  object        
 9   actual_price               410346 non-null  float64       
 10  yfinance_timestamp         410346 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(3), int64(1), object(4

In [59]:
# Convert timestamp columns to datetime
df['predicted_value_timestamp'] = pd.to_datetime(df['predicted_value_timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['yfinance_timestamp'] = pd.to_datetime(df['yfinance_timestamp'])

# Calculate prediction error
df['error'] = df['prediction'] - df['actual_price']

# Function to format timestamp range
def print_timestamp_range(column_name):
    earliest = df[column_name].min()
    latest = df[column_name].max()
    print(f"{column_name}:")
    print(f"  Earliest: {earliest}")
    print(f"  Latest: {latest}")
    print(f"  Range: {latest - earliest}")
    print()

# Print timestamp ranges
print("Timestamp Ranges:")
print_timestamp_range('predicted_value_timestamp')
print_timestamp_range('timestamp')
print_timestamp_range('yfinance_timestamp')

# Calculate and print mean error
mean_error = df["error"].mean()
print(f"Mean Error: {mean_error}")

# Additional error statistics
print("\nError Statistics:")
print(df["error"].describe())

# Calculate and print mean absolute error
mean_absolute_error = df["error"].abs().mean()
print(f"\nMean Absolute Error: {mean_absolute_error}")

# Calculate and print root mean squared error
root_mean_squared_error = (df["error"]**2).mean()**0.5
print(f"Root Mean Squared Error: {root_mean_squared_error}")

Timestamp Ranges:
predicted_value_timestamp:
  Earliest: 2024-09-19 13:40:00
  Latest: 2024-09-26 14:50:00
  Range: 7 days 01:10:00

timestamp:
  Earliest: 2024-09-19 13:35:00
  Latest: 2024-09-26 14:20:00
  Range: 7 days 00:45:00

yfinance_timestamp:
  Earliest: 2024-09-19 13:40:00
  Latest: 2024-09-26 14:50:00
  Range: 7 days 01:10:00

Mean Error: -0.2617058037197518

Error Statistics:
count    410346.000000
mean         -0.261706
std           7.287234
min        -107.227250
25%          -3.749512
50%          -0.004883
75%           3.418067
max         123.313290
Name: error, dtype: float64

Mean Absolute Error: 4.909307829035265
Root Mean Squared Error: 7.291922538213709


In [60]:

def calculate_delta_factor(group):
    actual_prices = group['actual_price'].values
    predictions = group['prediction'].values
    return pd.Series({
        'delta_factor': np.sum(np.abs(predictions - actual_prices)),
        'hot_key': group['hot_key'].iloc[0]  # Assuming hot_key is the same for all rows in the group
    })

# Assuming df is your DataFrame with columns: timestamp, miner_id, hot_key, minute_offset, prediction, actual_price
df_delta = df.groupby(['timestamp', 'miner_id']).apply(calculate_delta_factor).reset_index()

# Sort the results
df_delta_sorted = df_delta.sort_values(['timestamp', 'delta_factor'])

print(df_delta_sorted)

# If you want to calculate average delta factor per miner including hot_key
df_avg_delta = df_delta.groupby(['miner_id', 'hot_key'])['delta_factor'].mean().reset_index()
df_avg_delta_sorted = df_avg_delta.sort_values('delta_factor')

print("\nAverage delta factor per miner with hot_key (sorted, best performers first):")
print(df_avg_delta_sorted)

                timestamp                              miner_id  delta_factor  \
21    2024-09-19 13:35:00  22c79094-87a6-4930-a685-7e84ed4365d7     21.648139   
95    2024-09-19 13:35:00  93479517-87ae-4bda-8133-0b47b2f746e1     21.648139   
37    2024-09-19 13:35:00  3f9bb4d3-1d1d-4211-9478-dbd7cd3565a7     25.498779   
53    2024-09-19 13:35:00  55a91c7d-067c-4af9-83c1-8c11fbc8749f     27.401367   
82    2024-09-19 13:35:00  88250005-f7e7-49f2-9469-39b6c6de6acf     27.401367   
...                   ...                                   ...           ...   
68322 2024-09-26 14:20:00  b8c0bdd2-7964-439d-bd35-97f87d4947e8     80.928200   
68362 2024-09-26 14:20:00  e4017c59-245c-427d-bc8c-ea0bfe300f84     80.928200   
68380 2024-09-26 14:20:00  f098e2b4-7d26-432d-9620-2066f809b0b1     80.928200   
68383 2024-09-26 14:20:00  f55bebac-92b4-42e3-8d42-2c003d13fb12     80.928200   
68389 2024-09-26 14:20:00  fbb6e065-56aa-412f-b017-f6147cc2d3f2     80.928200   

                           

C:\Users\PeterCarlson\AppData\Local\Temp\ipykernel_28716\1689710648.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_delta = df.groupby(['timestamp', 'miner_id']).apply(calculate_delta_factor).reset_index()
